### Consigna del desafío 2

1. Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
2. Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
3. Graficarlos.
4. Obtener conclusiones.

#### Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec
import os
import platform
import nltk
nltk.download('punkt') 
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
from gensim.models.callbacks import CallbackAny2Vec

from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np     

import plotly.graph_objects as go
import plotly.express as px

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\julia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\julia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip   
else:
    print("El dataset ya se encuentra descargado")

El dataset ya se encuentra descargado


#### Parte 1

In [3]:
#Uso dataset de cansiones de michael jackson
df = pd.read_csv('songs_dataset/michael-jackson.txt', sep='/n', header=None)

C:\Users\julia\AppData\Local\Temp\ipykernel_32032\3874159358.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('songs_dataset/michael-jackson.txt', sep='/n', header=None)


In [4]:
sentence_tokens = []
for _, row in df[:None].iterrows():
    sentence_tokens.append(word_tokenize(row[0]))


In [5]:
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [6]:
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     sg=1) 

In [7]:
w2v_model.build_vocab(sentence_tokens)

In [8]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 9484


In [9]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 1192


---
### Parte 2
---

In [10]:
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=50,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 139328.140625
Loss after epoch 1: 106083.609375
Loss after epoch 2: 100470.5625
Loss after epoch 3: 96994.625
Loss after epoch 4: 86478.1875
Loss after epoch 5: 79767.5625
Loss after epoch 6: 76308.9375
Loss after epoch 7: 79980.3125
Loss after epoch 8: 69943.875
Loss after epoch 9: 67997.0
Loss after epoch 10: 64988.9375
Loss after epoch 11: 64643.0625
Loss after epoch 12: 64783.5625
Loss after epoch 13: 52734.375
Loss after epoch 14: 59545.75
Loss after epoch 15: 58708.75
Loss after epoch 16: 57941.75
Loss after epoch 17: 51108.0
Loss after epoch 18: 56889.75
Loss after epoch 19: 64026.875
Loss after epoch 20: 63299.875
Loss after epoch 21: 63689.5
Loss after epoch 22: 56169.25
Loss after epoch 23: 55668.75
Loss after epoch 24: 61938.5
Loss after epoch 25: 48464.75
Loss after epoch 26: 55150.875
Loss after epoch 27: 54567.125
Loss after epoch 28: 57019.625
Loss after epoch 29: 61259.25
Loss after epoch 30: 53116.375
Loss after epoch 31: 51988.5
Loss after epoch 32

(1946229, 2968650)

In [11]:
# Palabras que MÁS se relacionan con bad:
w2v_model.wv.most_similar(positive=["bad"], topn=10)

[('shameone', 0.7617164254188538),
 ('-', 0.6556829810142517),
 ('Dang', 0.6489359140396118),
 ('Raven', 0.6390835046768188),
 ('Bad', 0.6101269721984863),
 ('dah', 0.5751240849494934),
 ('Johnny', 0.5739648938179016),
 ('Gone', 0.570134699344635),
 ('Monster', 0.5692838430404663),
 ('Chorus', 0.5643842816352844)]

La mayor relación encontrada con la palabra **`bad`** es **`shameone`**. Esto es esperable, ya que dicha palabra se utiliza repetidamente en el estribillo de la canción, por lo cual es natural que ambos términos estén fuertemente vinculados. También aparece **`Bad`** (con mayúscula inicial), que es la misma palabra.

El resto de los términos similares podrían ser palabras que aparecen con frecuencia en proximidad a **`bad`**, tanto en la letra de esta canción como en la de otras.


In [12]:
w2v_model.wv.most_similar(negative=["bad"], topn=10)


[('door', -0.007609180640429258),
 ('From', -0.027203068137168884),
 ('must', -0.04648132249712944),
 ('tomorrow', -0.050977353006601334),
 ('Has', -0.051028620451688766),
 ('Will', -0.051464058458805084),
 ('Somebody', -0.05608798563480377),
 ('We', -0.07125600427389145),
 ('Wo', -0.0845637321472168),
 ('Into', -0.08582037687301636)]

Para las palabras que no se relacionan directamente con **`bad`**, se observa que parecen estar más asociadas a canciones con una connotación melancólica o de amor, por lo que no se encuentra un vínculo evidente con el término original.

In [13]:
# Palabras que MÁS se relacionan con feel:
w2v_model.wv.most_similar(positive=["feel"], topn=10)

[('shoulder', 0.49106964468955994),
 ('factual', 0.4813081622123718),
 ('hide', 0.47567102313041687),
 ('letting', 0.4368520975112915),
 ('cry', 0.43472033739089966),
 ('euphoria', 0.4346248507499695),
 ('Shameone', 0.4311755895614624),
 ('looked', 0.4306465685367584),
 ('matter', 0.42956113815307617),
 ('alright', 0.4273685812950134)]

Para el caso de la palabra **`feel`**, se encuentran términos con una muy buena relación, ya que todos ellos apuntan a cosas o sensaciones que alguien puede sentir. 

In [14]:
w2v_model.wv.most_similar(negative=["feel"], topn=10)


[('This', -0.01723630726337433),
 ('wo', -0.028111133724451065),
 ('Her', -0.0412001758813858),
 ('With', -0.04291465878486633),
 ('Of', -0.04603051394224167),
 ('into', -0.06254991143941879),
 ('Times', -0.06683771312236786),
 ('last', -0.06865434348583221),
 ('Your', -0.06878922134637833),
 ('Who', -0.07030123472213745)]

En este caso, los términos **no relacionados** con **`feel`** también parecen ser coherentes, ya que no son palabras que se utilizarían en el mismo contexto que la palabra base.

En resumen, para ambos casos se encontraron palabras que guardan relación con el término base utilizado, así como también se identificaron palabras sin ningún tipo de conexión aparente.


---
### Parte 3
---

In [15]:
def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [17]:
vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show() # esto para plotly en colab

---
### Parte 4
---

En general, se ha logrado observar el buen funcionamiento de Skip-Gram para encontrar palabras que guardan relación unas con otras; es decir, palabras que fácilmente son subsecuentes o que aparecen en contextos similares dentro de un texto, como en este caso, una canción.
